In [1]:
import utils

from minicons import scorer
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
model_name = "facebook/opt-125m"

if "gpt2" in model_name or "pythia" in model_name:
    bos_token = True
else:
    bos_token = False

lm = scorer.IncrementalLMScorer(model, device="cpu")

In [2]:
fk1999 = utils.read_csv_dict("../data/fk1999-final.csv")

In [3]:
# lm = scorer.IncrementalLMScorer("kanishka/smolm-autoreg-bpe-babylm-1e-3", device="cuda:0")
lm = scorer.IncrementalLMScorer("facebook/opt-6.7b", device="cuda:0")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dist = lm.next_word_distribution(fk1999[3]['prefix'])

In [5]:
fk1999[3]['prefix'], lm.topk(dist, 10)

("They told the little boy it was Bugs Bunny's favorite food. But he still didn't want to eat his",
 ([['vegetables',
    'carrots',
    'rabbit',
    'dinner',
    'broccoli',
    'lunch',
    'carrot',
    'favorite',
    'veggies',
    'peas']],
  [[0.10669860243797302,
    0.09968535602092743,
    0.04900696501135826,
    0.0432983860373497,
    0.03360813111066818,
    0.029452409595251083,
    0.02326599881052971,
    0.020190617069602013,
    0.019667377695441246,
    0.013422504998743534]]))

In [8]:
dist = lm.next_word_distribution([fk1999[11]['prefix'], fk1999[10]['prefix'], fk1999[20]['prefix']])

In [11]:
fk1999[20]

{'item': '21',
 'prefix': 'Everyone toasted the couple on fifty years of marriage. It was the first time some of the children there had ever tasted',
 'expected': 'champagne',
 'within_category': 'beer',
 'between_category': 'coffee',
 'cloze_expected': '0.607',
 'constraint': 'low'}

In [10]:
-1.0 * (dist * dist.exp()).sum(1)

tensor([4.1910, 6.0449, 4.7394])

In [6]:
# lm.query(dist, queries = [["jar", "box", "polar bear"], ["box", "jar", "button"]])
lm.query(dist, queries=[["panda", "lion", "polar bear"], ["box", "jar", "button"]], prob=False)

([[-15.695625305175781, -6.577995300292969, -15.695625305175781],
  [-5.581123352050781, -9.129615783691406, -9.67730712890625]],
 [[23432, 89, 23432], [36, 920, 1429]])

In [7]:
lm.conditional_score(fk1999[58]['prefix'], "polar bear", reduction=lambda x: x.sum().item())

[-5.922943115234375]

In [8]:
lm.token_score(fk1999[58]['prefix'] + " polar bear")

[[('H', 0.0),
  ('itting', -9.497499465942383),
  ('the', -1.4134979248046875),
  ('huge', -9.979400634765625),
  ('animal', -9.667678833007812),
  ('with', -2.8944244384765625),
  ('a', -1.373443603515625),
  ('tranquil', -7.095909118652344),
  ('izer', -0.0666961669921875),
  ('dart', -0.7112197875976562),
  ('was', -3.6857528686523438),
  ('difficult', -4.7639923095703125),
  ('in', -4.435646057128906),
  ('the', -1.4638900756835938),
  ('Arctic', -7.72998046875),
  ('winds', -9.886062622070312),
  ('.', -1.09564208984375),
  ('Eventually', -6.9127197265625),
  (',', -0.4878883361816406),
  ('however', -3.4646377563476562),
  (',', -0.0040283203125),
  ('they', -2.7923965454101562),
  ('were', -1.8530426025390625),
  ('able', -0.276702880859375),
  ('to', -0.00384521484375),
  ('approach', -7.191925048828125),
  ('and', -3.1918716430664062),
  ('tag', -6.98077392578125),
  ('the', -0.8248367309570312),
  ('polar', -5.741172790527344),
  ('bear', -0.18177032470703125)]]

In [17]:
0.5647839903831482 * 3.901865511579672e-06

2.203711173568351e-06